Installing the libraries

In [1]:
!pip install transformers datasets textract pypdf2 faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the libraries

In [3]:
import transformers
import datasets
import pandas as pd
import PyPDF2
from datasets import Dataset
import numpy as np
import faiss
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration, RagConfig
from transformers import AutoConfig, RagRetriever, BartConfig
from datasets import load_from_disk
from nltk.corpus import stopwords
import nltk
import string
from nltk.stem import WordNetLemmatizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
from transformers import RagTokenizer, BartTokenizer
from transformers import AutoModel

Importing the pdf file and removing first 7 pages as they were not useful

In [4]:
input_pdf_path = '/content/drive/MyDrive/Colab Notebooks/ugrulebook.pdf'
output_pdf_path = 'output.pdf'

pdf_reader = PyPDF2.PdfReader(open(input_pdf_path, 'rb'))

pdf_writer = PyPDF2.PdfWriter()

for page_num in range(7, len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    pdf_writer.add_page(page)

with open(output_pdf_path, 'wb') as output_file:
    pdf_writer.write(output_file)


Making a list of text which is not useful

In [5]:
forbidden_text = ["", " ", "  ", "   ", "    ", "     "]
for i in range(0,40):
  forbidden_text.append(str(i) + "." + " ")
  forbidden_text.append(str(i) + " ")
  forbidden_text.append(str(i))
  for j in range(0,40):
    forbidden_text.append(str(i) + "." + str(j) + " ")
    for k in range(0,40):
      forbidden_text.append(str(i) + "." + str(j) + "." + str(k) + " ")

Splitting the document into paragraphs

In [6]:
import re
import textract
text = textract.process(output_pdf_path).decode('utf-8')
splitted = re.split('\n',text)
splitted_final = [text for text in splitted if text not in forbidden_text]

Creating a dataframe

In [7]:
df2 = pd.DataFrame({'text': splitted_final})

Removing the paragraphs with less than three words

In [8]:
df = df2[df2['text'].apply(lambda x: len(x.split()) >= 3)]

In [9]:
df

,text
1,The B.Tech./ Dual Degree/ B.S. programmes con...
2,"sciences, engineering and technology and other..."
3,of three phases.
4,The first phase is an intense study of science...
5,of concepts than what was done in school.
...,...
1465,Consolidated statement of the Academic Perform...
1466,for all the semesters completed.
1468,: Under-Graduate Academic Performance Evaluat...
1469,: Under-Graduate Programmes Committee


Creating a text question generator pipeline

In [10]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="voidful/context-only-question-generator", device = 0)


Using nltk library for text processing such as tokenization, lemmatization etc.

In [11]:
!pip install nltk

In [12]:
from nltk.corpus import stopwords
import nltk

In [13]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
english_stopwords = stopwords.words('english')


In [15]:
df

,text
1,The B.Tech./ Dual Degree/ B.S. programmes con...
2,"sciences, engineering and technology and other..."
3,of three phases.
4,The first phase is an intense study of science...
5,of concepts than what was done in school.
...,...
1465,Consolidated statement of the Academic Perform...
1466,for all the semesters completed.
1468,: Under-Graduate Academic Performance Evaluat...
1469,: Under-Graduate Programmes Committee


In [16]:
def remove_punct(text):
    punct_free = "".join([i for i in text if i not in string.punctuation])
    return punct_free
df["title"] = df['text'].apply(lambda x: remove_punct(x))

<ipython-input-16-82bd3af23fa4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"] = df['text'].apply(lambda x: remove_punct(x))


In [17]:
from nltk.stem import WordNetLemmatizer
import string

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
word_net_lemmatizer = WordNetLemmatizer()

In [20]:
def tokenize_lemmatize(df):
    preprocessed = []
    for sen in df["title"]:
        tokens = sen.split()
        tokens = [word_net_lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in english_stopwords]
        sen1 = " ".join(tokens)
        preprocessed.append(sen1)
    df["title"] = preprocessed
    return df

In [21]:
df = tokenize_lemmatize(df)

<ipython-input-20-1be1a0735aa2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"] = preprocessed


In [22]:
df

,text,title
1,The B.Tech./ Dual Degree/ B.S. programmes con...,btech dual degree b programme consist course b...
2,"sciences, engineering and technology and other...",science engineering technology related topic s...
3,of three phases.,three phase
4,The first phase is an intense study of science...,first phase intense study science mathematics ...
5,of concepts than what was done in school.,concept done school
...,...,...
1465,Consolidated statement of the Academic Perform...,consolidated statement academic performance st...
1466,for all the semesters completed.,semester completed
1468,: Under-Graduate Academic Performance Evaluat...,undergraduate academic performance evaluation ...
1469,: Under-Graduate Programmes Committee,undergraduate programme committee


Generating questions out of text and adding them to a title column

In [23]:
df["title"] = df["title"].apply(lambda text: pipe(text, max_length=50, min_length=1, do_sample=False)[0]['generated_text'])

# Display the DataFrame with headings
print(df)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


                                                   text  \
1     The B.Tech./ Dual Degree/ B.S.  programmes con...   
2     sciences, engineering and technology and other...   
3                                    of three phases.     
4     The first phase is an intense study of science...   
5           of concepts than what was done in school.     
...                                                 ...   
1465  Consolidated statement of the Academic Perform...   
1466                  for all the semesters completed.    
1468  :  Under-Graduate Academic Performance Evaluat...   
1469            :  Under-Graduate Programmes Committee    
1470                  :  Undergraduate Research Award.    

                                                  title  
1              how to do btech dual degree b programme?  
2      Science engineering technology related topic ...  
3         how many phases are there in the three phase?  
4     the first phase intense study science math hum...  
5

<ipython-input-23-a4691b40c653>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"] = df["title"].apply(lambda text: pipe(text, max_length=50, min_length=1, do_sample=False)[0]['generated_text'])


In [24]:
from datasets import Dataset

In [25]:
df

,text,title
1,The B.Tech./ Dual Degree/ B.S. programmes con...,how to do btech dual degree b programme?
2,"sciences, engineering and technology and other...",Science engineering technology related topic ...
3,of three phases.,how many phases are there in the three phase?
4,The first phase is an intense study of science...,the first phase intense study science math hum...
5,of concepts than what was done in school.,did school have aconcept done school?
...,...,...
1465,Consolidated statement of the Academic Perform...,musical performance student
1466,for all the semesters completed.,"After themester was complete, what was the nex..."
1468,: Under-Graduate Academic Performance Evaluat...,undergraduate academic performance evaluation ...
1469,: Under-Graduate Programmes Committee,undergraduate program committee is comprised o...


Saving the processed dataset

In [26]:
dataset = Dataset.from_pandas(df)
dataset.save_to_disk("dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1227 [00:00<?, ? examples/s]

In [27]:
dataset_path = "dataset"  #Change the path accordingly
dataset = load_from_disk(dataset_path)

In [28]:
df = dataset.to_pandas()    #Converting the dataset to dataframe format

In [29]:
df

,text,title,__index_level_0__
0,The B.Tech./ Dual Degree/ B.S. programmes con...,how to do btech dual degree b programme?,1
1,"sciences, engineering and technology and other...",Science engineering technology related topic ...,2
2,of three phases.,how many phases are there in the three phase?,3
3,The first phase is an intense study of science...,the first phase intense study science math hum...,4
4,of concepts than what was done in school.,did school have aconcept done school?,5
...,...,...,...
1222,Consolidated statement of the Academic Perform...,musical performance student,1465
1223,for all the semesters completed.,"After themester was complete, what was the nex...",1466
1224,: Under-Graduate Academic Performance Evaluat...,undergraduate academic performance evaluation ...,1468
1225,: Under-Graduate Programmes Committee,undergraduate program committee is comprised o...,1469


In [30]:
word_net_lemmatizer = WordNetLemmatizer()   #Creating lemmatizer instance

In [31]:
df["preprocessed_text"] = df['text'].apply(lambda x: remove_punct(x))    #removing punctuations from each paragraph

In [32]:
df["preprocessed_text"]

0       The BTech Dual Degree BS  programmes consist o...
1       sciences engineering and technology and other ...
2                                       of three phases  
3       The first phase is an intense study of science...
4              of concepts than what was done in school  
                              ...                        
1222    Consolidated statement of the Academic Perform...
1223                     for all the semesters completed 
1224      UnderGraduate Academic Performance Evaluatio...
1225                  UnderGraduate Programmes Committee 
1226                        Undergraduate Research Award 
Name: preprocessed_text, Length: 1227, dtype: object

In [33]:
def tokenize_lemmatize(df):
    preprocessed = []
    for sen in df["preprocessed_text"]:
        tokens = sen.split()
        tokens = [word_net_lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in english_stopwords]
        sen1 = " ".join(tokens)
        preprocessed.append(sen1)
    df["preprocessed_text"] = preprocessed
    return df

In [34]:
df69 = tokenize_lemmatize(df)   #Tokenization and lemmatization and again joining the words to form lemmatized sentence

In [35]:
df69["preprocessed_text"]

0       btech dual degree b programme consist course b...
1       science engineering technology related topic s...
2                                             three phase
3       first phase intense study science mathematics ...
4                                     concept done school
                              ...                        
1222    consolidated statement academic performance st...
1223                                   semester completed
1224    undergraduate academic performance evaluation ...
1225                    undergraduate programme committee
1226                         undergraduate research award
Name: preprocessed_text, Length: 1227, dtype: object

In [36]:
torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [37]:
ds_with_embeddings = dataset.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["text"], return_tensors="pt"))[0][0].numpy()})
ds_with_embeddings.add_faiss_index(column='embeddings')

Map:   0%|          | 0/1227 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'title', '__index_level_0__', 'embeddings'],
    num_rows: 1227
})

In [38]:
ds_with_embeddings.save_faiss_index('embeddings', 'index3.faiss')   #Saving faiss indices
ds_with_embeddings.get_index("embeddings").save("index3.faiss")

In [39]:
df0 = ds_with_embeddings.to_pandas()
df90 = df0.drop(["__index_level_0__"], axis = 1)
ds_with_embeddings = Dataset.from_pandas(df90)
ds_with_embeddings.save_to_disk("Dataset3")   #Saving the processed dataset

Saving the dataset (0/1 shards):   0%|          | 0/1227 [00:00<?, ? examples/s]

Creating the RAG model using Hugging Face libraries

In [41]:
dataset_path = "Dataset3"    #Change the path accordingly to import processed dataset
index_path = "index3.faiss"  #Change the path accordingly to import faiss index


retriever_config = AutoConfig.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")    #Importing retriever configs
question_encoder_config = AutoConfig.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")   #Importing question encoder configs
generator_config = AutoConfig.from_pretrained("gpt2-large")   #Importing text generator model
retriever_name_or_path = "facebook/rag-sequence-base"    #Importing retriever

retriever = RagRetriever.from_pretrained(     #setting the retriever
    retriever_name_or_path=retriever_name_or_path,
    retriever_config=retriever_config,
    question_encoder_config=question_encoder_config,
    generator_config=generator_config,
    index_name="custom",
    passages_path=dataset_path,
    index_path=index_path,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [42]:
question_encoder = AutoModel.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")     #Setting the question encoder

Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base and are newly initialized: ['bert_model.encoder.layer.11.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.3.attention.self.value.bias', 'bert_model.encoder.layer.6.attention.self.key.weight', 'bert_model.encoder.layer.4.attention.self.value.weight', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.7.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.8.attention.self.query.weight', 'bert_model.encoder.layer.9.attention.self.query.bias', 'bert_model.encoder.layer.1.attention.self.value.weight', 'bert_model.encoder.layer.10.output.dense.bias', 'bert_model.encoder.layer.5.attention.output.dense.bias', 'bert_model.encoder.layer.8.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.10.output.LayerNorm.weight', 'bert_model.encoder.layer.11.intermediate.dense.bias', 'bert_model.encoder.layer.0.attention.sel

In [43]:

rag_config = RagConfig(     #Setting the configurations for RAG model
    dataset="custom",
    index_name="custom",
    index_path=index_path,  # Replace with the correct path
    passages_path=dataset_path,  # Replace with the correct path
    question_encoder={"model_type": "bart"},  # Use a compatible model for the question encoder
    generator={"model_type": "bart"},
)


In [44]:
from transformers import AutoModelForCausalLM

In [45]:
generator_model = AutoModelForCausalLM.from_pretrained("gpt2-large")    #Setting the generator model

In [46]:
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq",
                                                retriever=retriever,
                                                config=rag_config,
                                                question_encoder=question_encoder,
                                                tokenizer="facebook/rag-sequence-nq",)
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
model.generator = generator_model    #Setting the model's generator as the specified generator above

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [47]:
num_parameters = sum(p.numel() for p in model.parameters())
num_parameters

1289213184

Creating a function to take list of queries as input and give its answer as output

In [48]:
def query_to_answer(queries, model):
    outp = []
    for query in queries:
        input_dict = tokenizer.prepare_seq2seq_batch(query, padding=True, truncation=True, return_tensors="pt")
        generated = model.generate(input_ids=input_dict["input_ids"])
        outp.append(tokenizer.batch_decode(generated, skip_special_tokens=True)[0])
    return outp

Some example outputs

In [50]:
queries = ["What is ARP",
           "What is required number of credits to complete a minor",
           "What is compensatory time for PwD students",
           "What is Dean AA",
           "What is NP grade",
           "What is PP grade"]

In [51]:
answers = query_to_answer(queries, model)
answers

/usr/local/lib/python3.10/dist-packages/transformers/models/rag/tokenization_rag.py:87: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[' the academic rehabilitation program',
 ' 30',
 ' 20 minutes',
 ' academic dean',
 ' not pass',
 ' pass']

In [52]:
torch.save(model, 'FinalRAG-GPT2-LARGE.pth')

In [53]:
loaded_model = torch.load("FinalRAG-GPT2-LARGE.pth")

In [ ]:
answers = query_to_answer(queries, loaded_model)
answers

Importing Gradio and PyTorch

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch

Integrating the function to gradio interface to make the chatbot user friendly

In [ ]:
def process_input(text):
    text = [text]
    return query_to_answer(text, loaded_model)[0]

title = "QueryBOT"
examples = ["What is ARP",
           "What is required number of credits to complete a minor",
           "What is compensatory time for PwD students",
           "What is Dean AA",
           "What is NP grade",
           "What is PP grade"]

model_gui = gr.Interface(
    fn=process_input,
    inputs=gr.Textbox(lines=3, label="Input"),
    outputs=gr.Textbox(lines=3, label="Output"),
    title=title,
    examples=examples
)

model_gui.launch(share=True)